In [3]:
import numpy as np
user_matrice = np.load('user_matrice_filtered.npy') # 104 days x 57000 users x 3 weightings
user_stock_matrice = np.load('user_stock_matrice_filtered.npy') # 104 days x 78 stocks x 57000 users x 3 weightings
user_predictions = np.load('user_predictions.npy') # 104 days x 57000 users x 78 stocks
close_open_matrice = np.load('close_open_matrice.npy') # 104 x 78
print('done loading')


done loading


In [17]:
def findTotalUserWeightings(start_date, end_date, user_matrice, user_stock_matrice, weightings):
    # Match dimensions of user_stock matrics
    user_weighted = np.dot(user_matrice, weightings) # 104 x 57000
    repeated_user_weights = np.repeat(user_weighted[:,None], 78, axis=1) # 104 x 78 x 57000

    # Calculate user stock weights
    user_stock_weighted = np.dot(user_stock_matrice, weightings) # 104 x 78 x 57000

    total_user_weight = repeated_user_weights + user_stock_weighted # 104 x 78 x 57000

    return total_user_weight
    # print(total_user_weight.shape)

In [18]:
import datetime
start_date = datetime.datetime(2020, 1, 9)
end_date = datetime.datetime(2020, 6, 9)
weightings = np.array([1, 1, 1])

total_user_weight = findTotalUserWeightings(start_date, end_date, user_matrice, user_stock_matrice, weightings)

In [36]:
# normalize user predictions
count_total = np.count_nonzero(user_predictions, axis=1) # counts per day per stock
standardize_prediction = user_predictions / count_total[:,None]
standardize_prediction[np.isnan(standardize_prediction)] = 0

# Multiply user weights with user predictions
diagonal_weights = np.einsum('ijk,ikj->ij', total_user_weight, standardize_prediction) # 104 x 78
diagonal_weights[close_open_matrice<=0] = 0 # only keep top 20 stocks


In [37]:
top_n_stocks = 3
sorted_index = np.argsort(-abs(diagonal_weights),axis=1)
range_i = np.arange(diagonal_weights.shape[0])

# Find top stocks and corresponding close open
top_weights = diagonal_weights[range_i[:,None], sorted_index][:,:top_n_stocks]
top_close_open = close_open_matrice[range_i[:,None], sorted_index][:,:top_n_stocks]

In [41]:
# Find top n stocks to look at
top_n_stocks = 3
sorted_index = np.argsort(-abs(diagonal_weights),axis=1)
range_i = np.arange(diagonal_weights.shape[0])

# Find top stocks and corresponding close open
top_weights = diagonal_weights[range_i[:,None], sorted_index][:,:top_n_stocks]
top_close_open = close_open_matrice[range_i[:,None], sorted_index][:,:top_n_stocks]
total_weights = np.sum(abs(top_weights), axis=1)[:,None]
relative_weight = top_weights / total_weights
relative_weight[np.isnan(relative_weight)] = 0


for x in weighted_returns:
    print(relative_weight)

weighted_returns_perstock = np.multiply(relative_weight, top_close_open)
weighted_returns = np.sum(weighted_returns_perstock, axis=1)



-0.602101432326228
4.380786060113542
1.9172460458539085
3.7550957560161806
5.662472296180122
1.6928676859672183
4.3052710919523225
4.156554001952863
9.277939541693677
0.5414416961009231
7.265253319487492
0.925261953688536
1.0651805593707948
4.159361051770108
5.119667671703297
0.6594534316049039
1.8517611403185859
2.7299755679777977
0.8304156934314948
4.327292757054805
5.97046400706689
0.9510590879944205
1.1197002033692591
6.5392656547055665
0.7502000649257424
6.049755952863109
5.6617758742093
1.1952650582699562
9.02306809388907
2.638211364196104
3.8756410045919196
2.66008769172044
85.68518155415225
69.69916271217141
7.691674874816974
5.382893192363911
3.135252285603495
5.856046527892923
2.7644980253585496
19.7885731631795
3.1343050838421034
18.366769947645178
18.20777688920161
6.307874957523297
14.89361702127661
9.327521800153765
12.093841588380217
9.800128433893054
4.561850152035184
12.194140435822897
10.199402952524634
10.295737904215212
2.405689538165685
8.20094186608333
10.25938288